## Cost - Unit labor cost
## Efficiency - productivity

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

df = pd.read_csv("../data/Processed/efficiency_productivity/efficiency_productivity.csv")

# compute yearly average labor productivity and efficiency(producer)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df_year = df.groupby('Year', as_index=False)[['Labor productivity','Unit labor costs']].mean()
print(df_year)

    Year  Labor productivity  Unit labor costs
0   2013           102.84175          88.68700
1   2014           102.54325          91.29650
2   2015           100.76275          95.46550
3   2016           100.71300          96.18725
4   2017           100.00850          99.99750
5   2018           100.22675         102.18900
6   2019            97.90625         107.20625
7   2020            97.78950         114.53900
8   2021            99.48350         115.33175
9   2022            98.18900         120.61175
10  2023            98.03075         126.75750
11  2024            98.67175         130.99000


In [ ]:
# 

In [90]:
# draw core chart
ear_slider = alt.param(
    name='year_sel',
    bind=alt.binding_range(
        name='Show up to Year: ',
        min=int(df_year['Year'].min()),
        max=int(df_year['Year'].max()),
        step=1
    ),
    value=int(df_year['Year'].min())
)

base = alt.Chart(df_year).transform_filter("datum.Year <= year_sel").encode(
    x=alt.X('Labor productivity:Q', title='Labor Productivity (Index, 2017=100)', scale=alt.Scale(domain=[97.5, 103])),
    y=alt.Y('Unit labor costs:Q',  title='Unit Labor Costs (Index, 2017=100)', scale=alt.Scale(domain=[70, 140])),
    order=alt.Order('Year:O'),
    tooltip=[
      alt.Tooltip('Year:O', title='Year'),
      alt.Tooltip('Labor productivity:Q', format='.2f'),
      alt.Tooltip('Unit labor costs:Q', format='.2f'),
    ]
)

line   = base.mark_line(color='#1f77b4', strokeWidth=1)
points = base.mark_point(color='#ff7f0e', size=100, filled=True)

chart = (line + points).properties(
    width=800,
    title='Annual Connected Scatter: LP vs. ULC with Arrows'
).add_params(
    ear_slider
)

chart

alt.LayerChart(...)